### Webscraper for extracting newly published names, strains and accession numbers from the weeekly IJSEM email (saved in html). Script then compares the IJSEM names to the NCBI names and generates a report used for taxonomy updates. 
### Notes on selenium. There is a lot of bad advise in stack overflow that refers to old versions of selenium. Need to refer to the offocial documentation https://selenium-python.readthedocs.io/

In [1]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
#PATH = "C:\Users\mcveigh\Documents\PythonPC\chrome.exe"
#driver = webdriver.Chrome(PATH)
from selenium.webdriver.chrome.options import Options
import time
import pandas as pd
import re
import os
import sys
import bs4
from bs4 import BeautifulSoup
import requests
import numpy as np

### List of URLs input to search short list manually entered

In [ ]:
urls = ['https://www.microbiologyresearch.org/content/journal/ijsem/10.1099/ijsem.0.006404',
       'https://www.microbiologyresearch.org/content/journal/ijsem/10.1099/ijsem.0.006406?emailalert=true',
        'https://www.microbiologyresearch.org/content/journal/ijsem/10.1099/ijsem.0.006401?emailalert=true',
        'https://www.microbiologyresearch.org/content/journal/ijsem/10.1099/ijsem.0.006417?emailalert=true'
       ]

### Find URLS from saved email in html - save as from outlook in htm format

In [2]:
input = (r'IJSEMemail5.htm')

In [3]:
#with open ('IJSEMemail1.htm', encoding = 'unicode_escape') as f:
with open (input, encoding = 'unicode_escape') as f:
    content = f.read()
    soup = BeautifulSoup(content, 'html.parser')  
print(soup.prettify())

<html xmlns="http://www.w3.org/TR/REC-html40" xmlns:m="http://schemas.microsoft.com/office/2004/12/omml" xmlns:ns0="http://www.w3.org/1999/xhtml" xmlns:o="urn:schemas-microsoft-com:office:office" xmlns:v="urn:schemas-microsoft-com:vml" xmlns:w="urn:schemas-microsoft-com:office:word">
 <head>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <meta content="Word.Document" name="ProgId"/>
  <meta content="Microsoft Word 15" name="Generator"/>
  <meta content="Microsoft Word 15" name="Originator"/>
  <link href="IJSEMemail5_files/filelist.xml" rel="File-List"/>
  <link href="IJSEMemail5_files/editdata.mso" rel="Edit-Time-Data"/>
  <!--[if !mso]>
<style>
v\:* {behavior:url(#default#VML);}
o\:* {behavior:url(#default#VML);}
w\:* {behavior:url(#default#VML);}
.shape {behavior:url(#default#VML);}
</style>
<![endif]-->
  <!--[if gte mso 9]><xml>
 <o:OfficeDocumentSettings>
  <o:AllowPNG/>
 </o:OfficeDocumentSettings>
</xml><![endif]-->
  <link href="IJSEMemail5_files/them

In [4]:
urls = []
numberoflinks = 0
for url in soup.findAll( 'a', attrs={'href': re.compile("^https:")}):
    urls.append(url.get('href'))
    numberoflinks = numberoflinks + 1
    #print(url)
#print(numberoflinks)
#print(urls)
#remove_list = ['TandC','doi.org', 'myaccount']
urls = [e for e in urls if "TandC" not in e and "doi.org" not in e and "myaccount" not in e]

#print(dois)
#print(urls)

### Selenium web scraper - title and author note (footnote 1), some of these functions can be turned off following additional testing. Currently not using this function

In [5]:
#Function to scrape headlines using Selenium
def scrape_with_selenium(url, counter, doi):
    global title
    global footnote
    global description
    title = []
    description = None
    options = Options()
    options.headless = False  # Set to True for headless mode
    driver = webdriver.Chrome(options=options)

    #Navigate to the webpage
    driver.get(url)

    #Allow time for dynamic content to load (you may need to use WebDriverWait for more robust waiting)
    time.sleep(3)

    for element in driver.find_elements(By.CLASS_NAME, "item-meta-data__item-title"):
        #print(element.text)
        title = element.text
        #print(title)
    #for element in driver.find_elements(By.CLASS_NAME, "author-footnote-below-abstract"):get.text() #retrieves all footnotes
    for element in driver.find_elements(By.ID, "FN1"): #retrieves just footnote 1
        #print(element.text)
        footnote = element.text
        #print(footnote)
        #print(type(footnote))
        
    for element in driver.find_elements(By.PARTIAL_LINK_TEXT, "doi.org"):
        doi = element.text
        print(doi)
    
    for element in driver.find_elements(By.CSS_SELECTOR, "div.tl-main-part.title"): #finds section headers
        #print(element.text)
        counter += 1
        description = element.text
        print(description)
        if "Description of" in description: 
            print('found', description)  
            #snumber = 's' + str(counter - 4) + '/p[3]'
            snumber = 's' + str(counter - 4)
            print('snumber is', snumber)
            for element in driver.find_elements(By.ID, snumber):
                description = element.text
                #print(description)
        
    #Close the browser window
    driver.quit()
    return title, footnote, description, doi



### Beautifulsoup webscraper - returns abstract, can be turned off following additional testing. This function is called but we are not longer using the output.

In [7]:
def scrape_with_beautifulsoup(url):
    global abstract_text
    abstract_text = None
    response = requests.get(url)
    #soup = BeautifulSoup(response.content, "html.parser")
    soup = BeautifulSoup(response.text, "lxml")
    abstract = soup.find(class_= "articleabstract")
    if abstract is not None:
    #print(abstract.get_text())
        abstract_text = abstract.get_text()
        return abstract_text
    else:
        return

### Main body 

In [9]:
pub_df = pd.DataFrame(columns=['PublishedName', 'Accessions', 'Strains', 'DOI', 'URL'])
pd.set_option('display.max_columns', None)
for url in urls:
    counter = 1
    strains = []
    accessions = []
    doi = []
    description = None
    #scrape_with_selenium(url, counter)
    scrape_with_beautifulsoup(url)
    options = Options()
    options.headless = False  # Set to True for headless mode
    driver = webdriver.Chrome(options=options)

    #Navigate to the webpage
    driver.get(url)

    #Allow time for dynamic content to load (you may need to use WebDriverWait for more robust waiting)
    time.sleep(3)
    
    for element in driver.find_elements(By.CLASS_NAME, "item-meta-data__item-title"):
        #print(element.text)
        title = element.text
        print(title)
        
    for element in driver.find_elements(By.PARTIAL_LINK_TEXT, "doi.org"):
        doi = element.text
        print(doi)
        
    for element in driver.find_elements(By.CSS_SELECTOR, "div.tl-main-part.title"): #finds section headers
        #print(element.text)
        counter += 1
        description = element.text
        #print(description)
        if "Description of" in description: 
            #print('found', description)  
            #snumber = 's' + str(counter - 4) + '/p[3]'
            snumber = 's' + str(counter - 4)
            #print('snumber is', snumber)
            for element in driver.find_elements(By.ID, snumber):
                description = element.text
                #print(description)
        
                #find the organism names    
                match = [r'(\S+\s+){2}(?=sp. nov.)', r'(\S+\s+){2}(?=nom. nov.)']
                regex = re.compile(r'\b(' + '|'.join(match) + r')\b')
                orgname = [m.group() for m in regex.finditer(description)]
                print('orgname', orgname)

                #find the accessions
                pattern = [r'[A-Z]{2}\d{6}', r'[A-Z]{4}\d{8}', r'([A-Z]+)(_[A-Z]+)\d{6}', r'[A-Z]{6}\d{9}']
                regex = re.compile(r'\b(' + '|'.join(pattern) + r')\b')
                if description is not None:
                    accessions = [m.group() for m in regex.finditer(description)]
                    print('accessions', accessions)
    
                #find the strains
                #strainpattern = [r'(?<=type strain).*'] 
                strainpattern = [r'(?<=type strain).*?(?=\))']
                regex = re.compile(r'\b(' + '|'.join(strainpattern) + r')\b')
                if description is not None:
                    strains = [m.group() for m in regex.finditer(description)]
                    print('strain names', strains)
    
                #load data into pandas dataframe
                row_data = [orgname, accessions, strains, doi, url]
                length = len(pub_df)
                pub_df.loc[length] = row_data
            print('BREAK')
    
#Close the browser window
    driver.quit()    

Spartinivicinus poritis sp. nov., a red pigment-producing bacterium isolated from a scleractinian coral Porites lutea
https://doi.org/10.1099/ijsem.0.006444
orgname ['Spartinivicinus poritis ']
accessions ['OP852799', 'JAPMOU000000000']
strain names [', A2-2T (=MCCC 1K08228T=KCTC 8323T']
orgname ['Spartinivicinus poritis ']
accessions []
strain names []
BREAK
Chitinophaga pollutisoli sp. nov., isolated from contaminated sediment
https://doi.org/10.1099/ijsem.0.006447
orgname ['Chitinophaga pollutisoli ']
accessions ['OR493479', 'CP149822']
strain names [' is GPA1T (=KACC 23415T=JCM 36644T', ' are 6078\u2009kb and 53.8\u2009mol% (calculated from the whole genome sequence']
orgname ['Chitinophaga pollutisoli ']
accessions []
strain names []
BREAK
Elongatibacter sediminis gen. nov., sp. nov., isolated from intertidal sediment, and genomic comparison with all genera in the family Wenzhouxiangellaceae
https://doi.org/10.1099/ijsem.0.006440
orgname []
accessions []
strain names []
orgname []

In [10]:
pd.set_option('max_colwidth', None)
pub_df['Strains'] = [','.join(map(str, l)) for l in pub_df['Strains']]
pub_df

,PublishedName,Accessions,Strains,DOI,URL
0,[Spartinivicinus poritis ],"[OP852799, JAPMOU000000000]",", A2-2T (=MCCC 1K08228T=KCTC 8323T",https://doi.org/10.1099/ijsem.0.006444,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006444%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7C810f6efd0cea4a07a8a108dc9ce1ee88%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638557742630061852%7CUnknown%7CTWFpbGZsb3d8eyJWIjoiMC4wLjAwMDAiLCJQIjoiV2luMzIiLCJBTiI6Ik1haWwiLCJXVCI6Mn0%3D%7C60000%7C%7C%7C&sdata=9%2F0%2B1KfXpsjZDt8j%2BJb8ln5DxiI9lTm0UXfRAAg7cFM%3D&reserved=0
1,[Spartinivicinus poritis ],[],,https://doi.org/10.1099/ijsem.0.006444,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006444%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7C810f6efd0cea4a07a8a108dc9ce1ee88%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638557742630061852%7CUnknown%7CTWFpbGZsb3d8eyJWIjoiMC4wLjAwMDAiLCJQIjoiV2luMzIiLCJBTiI6Ik1haWwiLCJXVCI6Mn0%3D%7C60000%7C%7C%7C&sdata=9%2F0%2B1KfXpsjZDt8j%2BJb8ln5DxiI9lTm0UXfRAAg7cFM%3D&reserved=0
2,[Chitinophaga pollutisoli ],"[OR493479, CP149822]","is GPA1T (=KACC 23415T=JCM 36644T, are 6078 kb and 53.8 mol% (calculated from the whole genome sequence",https://doi.org/10.1099/ijsem.0.006447,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006447%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7C810f6efd0cea4a07a8a108dc9ce1ee88%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638557742630075250%7CUnknown%7CTWFpbGZsb3d8eyJWIjoiMC4wLjAwMDAiLCJQIjoiV2luMzIiLCJBTiI6Ik1haWwiLCJXVCI6Mn0%3D%7C60000%7C%7C%7C&sdata=I6FhgyfH86%2FnUJOec1%2F48VqXb4cFDwqIlFnJW9DjBi8%3D&reserved=0
3,[Chitinophaga pollutisoli ],[],,https://doi.org/10.1099/ijsem.0.006447,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006447%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7C810f6efd0cea4a07a8a108dc9ce1ee88%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638557742630075250%7CUnknown%7CTWFpbGZsb3d8eyJWIjoiMC4wLjAwMDAiLCJQIjoiV2luMzIiLCJBTiI6Ik1haWwiLCJXVCI6Mn0%3D%7C60000%7C%7C%7C&sdata=I6FhgyfH86%2FnUJOec1%2F48VqXb4cFDwqIlFnJW9DjBi8%3D&reserved=0
4,[],[],,https://doi.org/10.1099/ijsem.0.006440,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006440%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7C810f6efd0cea4a07a8a108dc9ce1ee88%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638557742630086110%7CUnknown%7CTWFpbGZsb3d8eyJWIjoiMC4wLjAwMDAiLCJQIjoiV2luMzIiLCJBTiI6Ik1haWwiLCJXVCI6Mn0%3D%7C60000%7C%7C%7C&sdata=CzvjHf1jz1d04ZQ%2B3cVrU1PnZPpE1Tm4kuZpTotxkLg%3D&reserved=0
5,[],[],,https://doi.org/10.1099/ijsem.0.006440,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006440%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7C810f6efd0cea4a07a8a108dc9ce1ee88%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638557742630086110%7CUnknown%7CTWFpbGZsb3d8eyJWIjoiMC4wLjAwMDAiLCJQIjoiV2luMzIiLCJBTiI6Ik1haWwiLCJXVCI6Mn0%3D%7C60000%7C%7C%7C&sdata=CzvjHf1jz1d04ZQ%2B3cVrU1PnZPpE1Tm4kuZpTotxkLg%3D&reserved=0
6,[Elongatibacter sediminis ],"[JAZHOG000000000, PP414254]",",, CH-27T (=MCCC 1H00480T=KCTC 8011T",https://doi.org/10.1099/ijsem.0.006440,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006440%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7C810f6efd0cea4a07a8a108dc9ce1ee88%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638557742630086110%7CUnknown%7CT

In [11]:
pub_df = pub_df.drop_duplicates(subset='PublishedName', keep="first")
pub_df

,PublishedName,Accessions,Strains,DOI,URL
0,[Spartinivicinus poritis ],"[OP852799, JAPMOU000000000]",", A2-2T (=MCCC 1K08228T=KCTC 8323T",https://doi.org/10.1099/ijsem.0.006444,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006444%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7C810f6efd0cea4a07a8a108dc9ce1ee88%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638557742630061852%7CUnknown%7CTWFpbGZsb3d8eyJWIjoiMC4wLjAwMDAiLCJQIjoiV2luMzIiLCJBTiI6Ik1haWwiLCJXVCI6Mn0%3D%7C60000%7C%7C%7C&sdata=9%2F0%2B1KfXpsjZDt8j%2BJb8ln5DxiI9lTm0UXfRAAg7cFM%3D&reserved=0
2,[Chitinophaga pollutisoli ],"[OR493479, CP149822]","is GPA1T (=KACC 23415T=JCM 36644T, are 6078 kb and 53.8 mol% (calculated from the whole genome sequence",https://doi.org/10.1099/ijsem.0.006447,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006447%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7C810f6efd0cea4a07a8a108dc9ce1ee88%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638557742630075250%7CUnknown%7CTWFpbGZsb3d8eyJWIjoiMC4wLjAwMDAiLCJQIjoiV2luMzIiLCJBTiI6Ik1haWwiLCJXVCI6Mn0%3D%7C60000%7C%7C%7C&sdata=I6FhgyfH86%2FnUJOec1%2F48VqXb4cFDwqIlFnJW9DjBi8%3D&reserved=0
4,[],[],,https://doi.org/10.1099/ijsem.0.006440,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006440%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7C810f6efd0cea4a07a8a108dc9ce1ee88%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638557742630086110%7CUnknown%7CTWFpbGZsb3d8eyJWIjoiMC4wLjAwMDAiLCJQIjoiV2luMzIiLCJBTiI6Ik1haWwiLCJXVCI6Mn0%3D%7C60000%7C%7C%7C&sdata=CzvjHf1jz1d04ZQ%2B3cVrU1PnZPpE1Tm4kuZpTotxkLg%3D&reserved=0
6,[Elongatibacter sediminis ],"[JAZHOG000000000, PP414254]",",, CH-27T (=MCCC 1H00480T=KCTC 8011T",https://doi.org/10.1099/ijsem.0.006440,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006440%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7C810f6efd0cea4a07a8a108dc9ce1ee88%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638557742630086110%7CUnknown%7CTWFpbGZsb3d8eyJWIjoiMC4wLjAwMDAiLCJQIjoiV2luMzIiLCJBTiI6Ik1haWwiLCJXVCI6Mn0%3D%7C60000%7C%7C%7C&sdata=CzvjHf1jz1d04ZQ%2B3cVrU1PnZPpE1Tm4kuZpTotxkLg%3D&reserved=0
8,[Streptococcus raffinosi ],"[OQ921791, JASUZV000000000]",is BME SL 6.1T (=VTCC 12812T=NBRC 116368T,https://doi.org/10.1099/ijsem.0.006442,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006442%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7C810f6efd0cea4a07a8a108dc9ce1ee88%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638557742630096737%7CUnknown%7CTWFpbGZsb3d8eyJWIjoiMC4wLjAwMDAiLCJQIjoiV2luMzIiLCJBTiI6Ik1haWwiLCJXVCI6Mn0%3D%7C60000%7C%7C%7C&sdata=O6b01BTYMr0ZptgaWYZF1l6gCoxG6kPEjDmr73GEQo4%3D&reserved=0
14,[Lacticaseibacillus parahuelsenbergensis ],[OR603137],", LD0937T, isolated from an Austrian grass silage, is deposited at the German Collection of Microorganisms (DSM 116105T, consists of a circular chromosome with a size of 3 055 873 bp and a G+C content of 48.0 mol%. The genome sequence, the 16S rRNA sequence, and the raw read data of strain DSM 116105T are deposited at GenBank and SRA (accession numbers: GCA_030770265.1, OR603137, PRJNA1002950",https://doi.org/10.6084/m9.figshare.26124445,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006441%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7C810f6efd0cea4a07a8a108dc9ce1ee88%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638557742630107390%7CUnknown%7CTWFpbGZsb3d8eyJWIjoiMC4wLjAwMDAiLCJQIjoiV2luMzI

In [12]:
#try drop duplicate accessions here
pub_df = pub_df.drop_duplicates(subset='Accessions', keep="first")
pub_df.explode(['PublishedName']).reset_index(drop=True)
pub_df

,PublishedName,Accessions,Strains,DOI,URL
0,[Spartinivicinus poritis ],"[OP852799, JAPMOU000000000]",", A2-2T (=MCCC 1K08228T=KCTC 8323T",https://doi.org/10.1099/ijsem.0.006444,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006444%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7C810f6efd0cea4a07a8a108dc9ce1ee88%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638557742630061852%7CUnknown%7CTWFpbGZsb3d8eyJWIjoiMC4wLjAwMDAiLCJQIjoiV2luMzIiLCJBTiI6Ik1haWwiLCJXVCI6Mn0%3D%7C60000%7C%7C%7C&sdata=9%2F0%2B1KfXpsjZDt8j%2BJb8ln5DxiI9lTm0UXfRAAg7cFM%3D&reserved=0
2,[Chitinophaga pollutisoli ],"[OR493479, CP149822]","is GPA1T (=KACC 23415T=JCM 36644T, are 6078 kb and 53.8 mol% (calculated from the whole genome sequence",https://doi.org/10.1099/ijsem.0.006447,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006447%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7C810f6efd0cea4a07a8a108dc9ce1ee88%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638557742630075250%7CUnknown%7CTWFpbGZsb3d8eyJWIjoiMC4wLjAwMDAiLCJQIjoiV2luMzIiLCJBTiI6Ik1haWwiLCJXVCI6Mn0%3D%7C60000%7C%7C%7C&sdata=I6FhgyfH86%2FnUJOec1%2F48VqXb4cFDwqIlFnJW9DjBi8%3D&reserved=0
4,[],[],,https://doi.org/10.1099/ijsem.0.006440,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006440%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7C810f6efd0cea4a07a8a108dc9ce1ee88%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638557742630086110%7CUnknown%7CTWFpbGZsb3d8eyJWIjoiMC4wLjAwMDAiLCJQIjoiV2luMzIiLCJBTiI6Ik1haWwiLCJXVCI6Mn0%3D%7C60000%7C%7C%7C&sdata=CzvjHf1jz1d04ZQ%2B3cVrU1PnZPpE1Tm4kuZpTotxkLg%3D&reserved=0
6,[Elongatibacter sediminis ],"[JAZHOG000000000, PP414254]",",, CH-27T (=MCCC 1H00480T=KCTC 8011T",https://doi.org/10.1099/ijsem.0.006440,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006440%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7C810f6efd0cea4a07a8a108dc9ce1ee88%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638557742630086110%7CUnknown%7CTWFpbGZsb3d8eyJWIjoiMC4wLjAwMDAiLCJQIjoiV2luMzIiLCJBTiI6Ik1haWwiLCJXVCI6Mn0%3D%7C60000%7C%7C%7C&sdata=CzvjHf1jz1d04ZQ%2B3cVrU1PnZPpE1Tm4kuZpTotxkLg%3D&reserved=0
8,[Streptococcus raffinosi ],"[OQ921791, JASUZV000000000]",is BME SL 6.1T (=VTCC 12812T=NBRC 116368T,https://doi.org/10.1099/ijsem.0.006442,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006442%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7C810f6efd0cea4a07a8a108dc9ce1ee88%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638557742630096737%7CUnknown%7CTWFpbGZsb3d8eyJWIjoiMC4wLjAwMDAiLCJQIjoiV2luMzIiLCJBTiI6Ik1haWwiLCJXVCI6Mn0%3D%7C60000%7C%7C%7C&sdata=O6b01BTYMr0ZptgaWYZF1l6gCoxG6kPEjDmr73GEQo4%3D&reserved=0
14,[Lacticaseibacillus parahuelsenbergensis ],[OR603137],", LD0937T, isolated from an Austrian grass silage, is deposited at the German Collection of Microorganisms (DSM 116105T, consists of a circular chromosome with a size of 3 055 873 bp and a G+C content of 48.0 mol%. The genome sequence, the 16S rRNA sequence, and the raw read data of strain DSM 116105T are deposited at GenBank and SRA (accession numbers: GCA_030770265.1, OR603137, PRJNA1002950",https://doi.org/10.6084/m9.figshare.26124445,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006441%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7C810f6efd0cea4a07a8a108dc9ce1ee88%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638557742630107390%7CUnknown%7CTWFpbGZsb3d8eyJWIjoiMC4wLjAwMDAiLCJQIjoiV2luMzI

In [13]:
pub2_df = pub_df.explode(['Accessions']).reset_index(drop=True)
pub2_df

,PublishedName,Accessions,Strains,DOI,URL
0,[Spartinivicinus poritis ],OP852799,", A2-2T (=MCCC 1K08228T=KCTC 8323T",https://doi.org/10.1099/ijsem.0.006444,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006444%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7C810f6efd0cea4a07a8a108dc9ce1ee88%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638557742630061852%7CUnknown%7CTWFpbGZsb3d8eyJWIjoiMC4wLjAwMDAiLCJQIjoiV2luMzIiLCJBTiI6Ik1haWwiLCJXVCI6Mn0%3D%7C60000%7C%7C%7C&sdata=9%2F0%2B1KfXpsjZDt8j%2BJb8ln5DxiI9lTm0UXfRAAg7cFM%3D&reserved=0
1,[Spartinivicinus poritis ],JAPMOU000000000,", A2-2T (=MCCC 1K08228T=KCTC 8323T",https://doi.org/10.1099/ijsem.0.006444,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006444%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7C810f6efd0cea4a07a8a108dc9ce1ee88%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638557742630061852%7CUnknown%7CTWFpbGZsb3d8eyJWIjoiMC4wLjAwMDAiLCJQIjoiV2luMzIiLCJBTiI6Ik1haWwiLCJXVCI6Mn0%3D%7C60000%7C%7C%7C&sdata=9%2F0%2B1KfXpsjZDt8j%2BJb8ln5DxiI9lTm0UXfRAAg7cFM%3D&reserved=0
2,[Chitinophaga pollutisoli ],OR493479,"is GPA1T (=KACC 23415T=JCM 36644T, are 6078 kb and 53.8 mol% (calculated from the whole genome sequence",https://doi.org/10.1099/ijsem.0.006447,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006447%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7C810f6efd0cea4a07a8a108dc9ce1ee88%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638557742630075250%7CUnknown%7CTWFpbGZsb3d8eyJWIjoiMC4wLjAwMDAiLCJQIjoiV2luMzIiLCJBTiI6Ik1haWwiLCJXVCI6Mn0%3D%7C60000%7C%7C%7C&sdata=I6FhgyfH86%2FnUJOec1%2F48VqXb4cFDwqIlFnJW9DjBi8%3D&reserved=0
3,[Chitinophaga pollutisoli ],CP149822,"is GPA1T (=KACC 23415T=JCM 36644T, are 6078 kb and 53.8 mol% (calculated from the whole genome sequence",https://doi.org/10.1099/ijsem.0.006447,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006447%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7C810f6efd0cea4a07a8a108dc9ce1ee88%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638557742630075250%7CUnknown%7CTWFpbGZsb3d8eyJWIjoiMC4wLjAwMDAiLCJQIjoiV2luMzIiLCJBTiI6Ik1haWwiLCJXVCI6Mn0%3D%7C60000%7C%7C%7C&sdata=I6FhgyfH86%2FnUJOec1%2F48VqXb4cFDwqIlFnJW9DjBi8%3D&reserved=0
4,[],NaN,,https://doi.org/10.1099/ijsem.0.006440,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006440%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7C810f6efd0cea4a07a8a108dc9ce1ee88%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638557742630086110%7CUnknown%7CTWFpbGZsb3d8eyJWIjoiMC4wLjAwMDAiLCJQIjoiV2luMzIiLCJBTiI6Ik1haWwiLCJXVCI6Mn0%3D%7C60000%7C%7C%7C&sdata=CzvjHf1jz1d04ZQ%2B3cVrU1PnZPpE1Tm4kuZpTotxkLg%3D&reserved=0
5,[Elongatibacter sediminis ],JAZHOG000000000,",, CH-27T (=MCCC 1H00480T=KCTC 8011T",https://doi.org/10.1099/ijsem.0.006440,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006440%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7C810f6efd0cea4a07a8a108dc9ce1ee88%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638557742630086110%7CUnknown%7CTWFpbGZsb3d8eyJWIjoiMC4wLjAwMDAiLCJQIjoiV2luMzIiLCJBTiI6Ik1haWwiLCJXVCI6Mn0%3D%7C60000%7C%7C%7C&sdata=CzvjHf1jz1d04ZQ%2B3cVrU1PnZPpE1Tm4kuZpTotxkLg%3D&reserved=0
6,[Elongatibacter sediminis ],PP414254,",, CH-27T (=MCCC 1H00480T=KCTC 8011T",https://doi.org/10.1099/ijsem.0.006440,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.

In [14]:
pub4_df = pub2_df.explode(['PublishedName']).reset_index(drop=True)
pub4_df.rename(columns={'Accessions' : 'accession'}, inplace=True)
#pub4_df = pub4_df.dropna()
#pub4_df = pub4_df.drop_duplicates(subset='accession', keep="first")
pub4_df=pub4_df[pub4_df['accession'].isnull() | ~pub4_df[pub4_df['accession'].notnull()].duplicated(subset='accession',keep='first')]
pub4_df

,PublishedName,accession,Strains,DOI,URL
0,Spartinivicinus poritis,OP852799,", A2-2T (=MCCC 1K08228T=KCTC 8323T",https://doi.org/10.1099/ijsem.0.006444,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006444%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7C810f6efd0cea4a07a8a108dc9ce1ee88%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638557742630061852%7CUnknown%7CTWFpbGZsb3d8eyJWIjoiMC4wLjAwMDAiLCJQIjoiV2luMzIiLCJBTiI6Ik1haWwiLCJXVCI6Mn0%3D%7C60000%7C%7C%7C&sdata=9%2F0%2B1KfXpsjZDt8j%2BJb8ln5DxiI9lTm0UXfRAAg7cFM%3D&reserved=0
1,Spartinivicinus poritis,JAPMOU000000000,", A2-2T (=MCCC 1K08228T=KCTC 8323T",https://doi.org/10.1099/ijsem.0.006444,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006444%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7C810f6efd0cea4a07a8a108dc9ce1ee88%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638557742630061852%7CUnknown%7CTWFpbGZsb3d8eyJWIjoiMC4wLjAwMDAiLCJQIjoiV2luMzIiLCJBTiI6Ik1haWwiLCJXVCI6Mn0%3D%7C60000%7C%7C%7C&sdata=9%2F0%2B1KfXpsjZDt8j%2BJb8ln5DxiI9lTm0UXfRAAg7cFM%3D&reserved=0
2,Chitinophaga pollutisoli,OR493479,"is GPA1T (=KACC 23415T=JCM 36644T, are 6078 kb and 53.8 mol% (calculated from the whole genome sequence",https://doi.org/10.1099/ijsem.0.006447,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006447%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7C810f6efd0cea4a07a8a108dc9ce1ee88%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638557742630075250%7CUnknown%7CTWFpbGZsb3d8eyJWIjoiMC4wLjAwMDAiLCJQIjoiV2luMzIiLCJBTiI6Ik1haWwiLCJXVCI6Mn0%3D%7C60000%7C%7C%7C&sdata=I6FhgyfH86%2FnUJOec1%2F48VqXb4cFDwqIlFnJW9DjBi8%3D&reserved=0
3,Chitinophaga pollutisoli,CP149822,"is GPA1T (=KACC 23415T=JCM 36644T, are 6078 kb and 53.8 mol% (calculated from the whole genome sequence",https://doi.org/10.1099/ijsem.0.006447,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006447%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7C810f6efd0cea4a07a8a108dc9ce1ee88%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638557742630075250%7CUnknown%7CTWFpbGZsb3d8eyJWIjoiMC4wLjAwMDAiLCJQIjoiV2luMzIiLCJBTiI6Ik1haWwiLCJXVCI6Mn0%3D%7C60000%7C%7C%7C&sdata=I6FhgyfH86%2FnUJOec1%2F48VqXb4cFDwqIlFnJW9DjBi8%3D&reserved=0
4,NaN,NaN,,https://doi.org/10.1099/ijsem.0.006440,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006440%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7C810f6efd0cea4a07a8a108dc9ce1ee88%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638557742630086110%7CUnknown%7CTWFpbGZsb3d8eyJWIjoiMC4wLjAwMDAiLCJQIjoiV2luMzIiLCJBTiI6Ik1haWwiLCJXVCI6Mn0%3D%7C60000%7C%7C%7C&sdata=CzvjHf1jz1d04ZQ%2B3cVrU1PnZPpE1Tm4kuZpTotxkLg%3D&reserved=0
5,Elongatibacter sediminis,JAZHOG000000000,",, CH-27T (=MCCC 1H00480T=KCTC 8011T",https://doi.org/10.1099/ijsem.0.006440,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006440%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7C810f6efd0cea4a07a8a108dc9ce1ee88%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638557742630086110%7CUnknown%7CTWFpbGZsb3d8eyJWIjoiMC4wLjAwMDAiLCJQIjoiV2luMzIiLCJBTiI6Ik1haWwiLCJXVCI6Mn0%3D%7C60000%7C%7C%7C&sdata=CzvjHf1jz1d04ZQ%2B3cVrU1PnZPpE1Tm4kuZpTotxkLg%3D&reserved=0
6,Elongatibacter sediminis,PP414254,",, CH-27T (=MCCC 1H00480T=KCTC 8011T",https://doi.org/10.1099/ijsem.0.006440,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006440%3Femailaler

In [15]:
df_unique= pub4_df.drop_duplicates(["accession"], keep="first")
df_unique = df_unique.dropna()
df_unique

,PublishedName,accession,Strains,DOI,URL
0,Spartinivicinus poritis,OP852799,", A2-2T (=MCCC 1K08228T=KCTC 8323T",https://doi.org/10.1099/ijsem.0.006444,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006444%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7C810f6efd0cea4a07a8a108dc9ce1ee88%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638557742630061852%7CUnknown%7CTWFpbGZsb3d8eyJWIjoiMC4wLjAwMDAiLCJQIjoiV2luMzIiLCJBTiI6Ik1haWwiLCJXVCI6Mn0%3D%7C60000%7C%7C%7C&sdata=9%2F0%2B1KfXpsjZDt8j%2BJb8ln5DxiI9lTm0UXfRAAg7cFM%3D&reserved=0
1,Spartinivicinus poritis,JAPMOU000000000,", A2-2T (=MCCC 1K08228T=KCTC 8323T",https://doi.org/10.1099/ijsem.0.006444,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006444%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7C810f6efd0cea4a07a8a108dc9ce1ee88%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638557742630061852%7CUnknown%7CTWFpbGZsb3d8eyJWIjoiMC4wLjAwMDAiLCJQIjoiV2luMzIiLCJBTiI6Ik1haWwiLCJXVCI6Mn0%3D%7C60000%7C%7C%7C&sdata=9%2F0%2B1KfXpsjZDt8j%2BJb8ln5DxiI9lTm0UXfRAAg7cFM%3D&reserved=0
2,Chitinophaga pollutisoli,OR493479,"is GPA1T (=KACC 23415T=JCM 36644T, are 6078 kb and 53.8 mol% (calculated from the whole genome sequence",https://doi.org/10.1099/ijsem.0.006447,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006447%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7C810f6efd0cea4a07a8a108dc9ce1ee88%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638557742630075250%7CUnknown%7CTWFpbGZsb3d8eyJWIjoiMC4wLjAwMDAiLCJQIjoiV2luMzIiLCJBTiI6Ik1haWwiLCJXVCI6Mn0%3D%7C60000%7C%7C%7C&sdata=I6FhgyfH86%2FnUJOec1%2F48VqXb4cFDwqIlFnJW9DjBi8%3D&reserved=0
3,Chitinophaga pollutisoli,CP149822,"is GPA1T (=KACC 23415T=JCM 36644T, are 6078 kb and 53.8 mol% (calculated from the whole genome sequence",https://doi.org/10.1099/ijsem.0.006447,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006447%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7C810f6efd0cea4a07a8a108dc9ce1ee88%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638557742630075250%7CUnknown%7CTWFpbGZsb3d8eyJWIjoiMC4wLjAwMDAiLCJQIjoiV2luMzIiLCJBTiI6Ik1haWwiLCJXVCI6Mn0%3D%7C60000%7C%7C%7C&sdata=I6FhgyfH86%2FnUJOec1%2F48VqXb4cFDwqIlFnJW9DjBi8%3D&reserved=0
5,Elongatibacter sediminis,JAZHOG000000000,",, CH-27T (=MCCC 1H00480T=KCTC 8011T",https://doi.org/10.1099/ijsem.0.006440,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006440%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7C810f6efd0cea4a07a8a108dc9ce1ee88%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638557742630086110%7CUnknown%7CTWFpbGZsb3d8eyJWIjoiMC4wLjAwMDAiLCJQIjoiV2luMzIiLCJBTiI6Ik1haWwiLCJXVCI6Mn0%3D%7C60000%7C%7C%7C&sdata=CzvjHf1jz1d04ZQ%2B3cVrU1PnZPpE1Tm4kuZpTotxkLg%3D&reserved=0
6,Elongatibacter sediminis,PP414254,",, CH-27T (=MCCC 1H00480T=KCTC 8011T",https://doi.org/10.1099/ijsem.0.006440,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006440%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7C810f6efd0cea4a07a8a108dc9ce1ee88%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638557742630086110%7CUnknown%7CTWFpbGZsb3d8eyJWIjoiMC4wLjAwMDAiLCJQIjoiV2luMzIiLCJBTiI6Ik1haWwiLCJXVCI6Mn0%3D%7C60000%7C%7C%7C&sdata=CzvjHf1jz1d04ZQ%2B3cVrU1PnZPpE1Tm4kuZpTotxkLg%3D&reserved=0
7,Streptococcus raffinosi,OQ921791,is BME SL 6.1T (=VTCC 12812T=NBRC 116368T,https://doi.org/10.1099/ijsem.0.006442,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%

### Create a dataframe of unique accessions and look up NCBI taxonomy information of each accession with srcchk

In [ ]:
#df_unique.dtypes

In [16]:
df_unique['accession'] = df_unique['accession'].astype('str') 

In [17]:
with open('acclist', 'w') as f:
    for text in df_unique['accession'].tolist():
        f.write(text + '\n')

In [18]:
os.system("/netopt/ncbi_tools64/bin/srcchk -i acclist -f taxname,taxid,strain -o acclist.taxdata")


0

In [19]:
taxdata_file_name = (r'acclist.taxdata')    
srcchk_df = pd.read_csv(taxdata_file_name, sep='\t', index_col=None, low_memory=False)
srcchk_df.drop(columns=['Unnamed: 4'], inplace=True)
srcchk_df.rename(columns={'organism' : 'NCBIname'}, inplace=True)
srcchk_df['accession'] = srcchk_df['accession'].astype(str).replace('\.\d+', '', regex=True).astype(str)
srcchk_df = srcchk_df.dropna()
srcchk_df 
#many need to add dropna here

,accession,NCBIname,taxid,strain
0,OP852799,Spartinivicinus sp.,2768740,A2-2
1,JAPMOU000000000,Spartinivicinus sp. A2-2,2994640,A2-2
2,OR493479,Chitinophaga sp.,1869181,GPA1
3,CP149822,Chitinophaga sp. GPA1,3133966,GPA1
4,JAZHOG000000000,Wenzhouxiangellaceae bacterium CH-27,3119006,CH-27
5,PP414254,Wenzhouxiangella sp.,1979961,CH-27
6,OQ921791,Streptococcus sp.,1306,VTCC 12812
7,JASUZV000000000,Streptococcus sp. VTCC 12812,3053355,VTCC 12812
8,OR603137,Lacticaseibacillus sp.,2775876,NCIMB 15471
9,OR603138,Lacticaseibacillus sp.,2775876,NCIMB 15473


### Combine dataframes into one

In [21]:
combine_df=pd.merge(left=pub4_df, right=srcchk_df, left_on='accession', right_on='accession', how = 'outer')
combine_df = combine_df[['PublishedName', 'NCBIname', 'Strains', 'accession', 'strain', 'taxid', 'DOI', 'URL' ]]
combine_df

,PublishedName,NCBIname,Strains,accession,strain,taxid,DOI,URL
0,Spartinivicinus poritis,Spartinivicinus sp.,", A2-2T (=MCCC 1K08228T=KCTC 8323T",OP852799,A2-2,2768740.0,https://doi.org/10.1099/ijsem.0.006444,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006444%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7C810f6efd0cea4a07a8a108dc9ce1ee88%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638557742630061852%7CUnknown%7CTWFpbGZsb3d8eyJWIjoiMC4wLjAwMDAiLCJQIjoiV2luMzIiLCJBTiI6Ik1haWwiLCJXVCI6Mn0%3D%7C60000%7C%7C%7C&sdata=9%2F0%2B1KfXpsjZDt8j%2BJb8ln5DxiI9lTm0UXfRAAg7cFM%3D&reserved=0
1,Spartinivicinus poritis,Spartinivicinus sp. A2-2,", A2-2T (=MCCC 1K08228T=KCTC 8323T",JAPMOU000000000,A2-2,2994640.0,https://doi.org/10.1099/ijsem.0.006444,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006444%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7C810f6efd0cea4a07a8a108dc9ce1ee88%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638557742630061852%7CUnknown%7CTWFpbGZsb3d8eyJWIjoiMC4wLjAwMDAiLCJQIjoiV2luMzIiLCJBTiI6Ik1haWwiLCJXVCI6Mn0%3D%7C60000%7C%7C%7C&sdata=9%2F0%2B1KfXpsjZDt8j%2BJb8ln5DxiI9lTm0UXfRAAg7cFM%3D&reserved=0
2,Chitinophaga pollutisoli,Chitinophaga sp.,"is GPA1T (=KACC 23415T=JCM 36644T, are 6078 kb and 53.8 mol% (calculated from the whole genome sequence",OR493479,GPA1,1869181.0,https://doi.org/10.1099/ijsem.0.006447,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006447%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7C810f6efd0cea4a07a8a108dc9ce1ee88%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638557742630075250%7CUnknown%7CTWFpbGZsb3d8eyJWIjoiMC4wLjAwMDAiLCJQIjoiV2luMzIiLCJBTiI6Ik1haWwiLCJXVCI6Mn0%3D%7C60000%7C%7C%7C&sdata=I6FhgyfH86%2FnUJOec1%2F48VqXb4cFDwqIlFnJW9DjBi8%3D&reserved=0
3,Chitinophaga pollutisoli,Chitinophaga sp. GPA1,"is GPA1T (=KACC 23415T=JCM 36644T, are 6078 kb and 53.8 mol% (calculated from the whole genome sequence",CP149822,GPA1,3133966.0,https://doi.org/10.1099/ijsem.0.006447,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006447%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7C810f6efd0cea4a07a8a108dc9ce1ee88%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638557742630075250%7CUnknown%7CTWFpbGZsb3d8eyJWIjoiMC4wLjAwMDAiLCJQIjoiV2luMzIiLCJBTiI6Ik1haWwiLCJXVCI6Mn0%3D%7C60000%7C%7C%7C&sdata=I6FhgyfH86%2FnUJOec1%2F48VqXb4cFDwqIlFnJW9DjBi8%3D&reserved=0
4,NaN,NaN,,NaN,NaN,NaN,https://doi.org/10.1099/ijsem.0.006440,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006440%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7C810f6efd0cea4a07a8a108dc9ce1ee88%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638557742630086110%7CUnknown%7CTWFpbGZsb3d8eyJWIjoiMC4wLjAwMDAiLCJQIjoiV2luMzIiLCJBTiI6Ik1haWwiLCJXVCI6Mn0%3D%7C60000%7C%7C%7C&sdata=CzvjHf1jz1d04ZQ%2B3cVrU1PnZPpE1Tm4kuZpTotxkLg%3D&reserved=0
5,Elongatibacter sediminis,Wenzhouxiangellaceae bacterium CH-27,",, CH-27T (=MCCC 1H00480T=KCTC 8011T",JAZHOG000000000,CH-27,3119006.0,https://doi.org/10.1099/ijsem.0.006440,https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fwww.microbiologyresearch.org%2Fcontent%2Fjournal%2Fijsem%2F10.1099%2Fijsem.0.006440%3Femailalert%3Dtrue&data=05%7C02%7Cmcveigh%40ncbi.nlm.nih.gov%7C810f6efd0cea4a07a8a108dc9ce1ee88%7C14b77578977342d58507251ca2dc2b06%7C0%7C0%7C638557742630086110%7CUnknown%7CTWFpbGZsb3d8eyJWIjoiMC4wLjAwMDAiLCJQIjoiV2luMzIiLCJBTiI6Ik1haWwiLCJXVCI6Mn0%3D%7C60000%7C%7C%7C&sdata=CzvjHf1jz1d04ZQ%2B3cVrU1PnZPpE1Tm4kuZpTotxkLg%3D&reserved=0
6,Elongatibacter sediminis,Wenzhouxiangella 

### write output to excel

In [22]:
combine_df.to_excel('NameCheckweek5b.xlsx', engine='xlsxwriter', index = False, na_rep = '') 

### taxonerd test -- skip this

In [ ]:
from taxonerd import TaxoNERD
taxonerd = TaxoNERD(prefer_gpu=False)
nlp = taxonerd.load(model="en_ner_eco_md", exclude=[], linker="taxref", threshold=0.7)
nlp.pipe_names

### option to use biopython to retrieve orgnames based on accessions -- this approarch works but using srcchk for now so this isn't needed

In [ ]:
import Bio
from Bio import Entrez
from Bio import SeqIO

In [ ]:
handle = Entrez.efetch(db="nucleotide", id="AY851612", rettype="gb", retmode="text")
x = SeqIO.read(handle, 'genbank')
x.annotations['organism']

### extra stuff not using, just saving for now

In [ ]:
combine_df['boo'] = combine_df['PublishedName'] != combine_df['NCBIname']

combine_df.style.applymap('background-color: red', subset=['PublishedName'])

combine_df

In [ ]:
 def color_positive_green(val):
    if combine_df.iloc['boo'] == True:
        color = 'red'
    else:
        color = 'black'
    return 'color: %s' % color
 
combine_df.style.applymap(color_positive_green)

In [ ]:
#combine_df.apply_style_by_indexes(combine_df[combine_df['PublishedName'] != combine_df['NCBIname']], styler_obj=Styler(bg_color='red'), cols_to_style=['PublishedName', 'NCBIname'])

In [ ]:
#df.style.apply(color, axis=None).to_excel('styled.xlsx', engine='openpyxl')